### Lógica:
- Extrair a lista de CEPs a partir de um dataframe e remover registros duplicados
- Buscar no database se um registro de CEP com localidade já existe (consumir menos dados da API e otimizar processamento)
- Se o registro existir, pular para o próximo CEP
- Se o registro não existir, consultar a localidade utilizando API externa
- Salvar o registro no DB e continuar a busca, até finalizar.
- Realizar a união dos valores de localidade do DB com os CEPs das localidades do relatório do BDO
- Reorganizar ordem das colunas do relatório e eliminar colunas inválidas
- Aplicar filtros de acordo com as regras de negócio e salvar as saídas em arquivos csv.

### Este jupyter notebook é responsável por agregar as funções e executar o código de forma mais clara ao usuário. Apesar disso, melhorias são necessárias para torná-lo mais legível.

In [1]:
#Esta função é responsável por carregar o arquivo CSV extraído do BDO e filtrar todos os CEPs e eliminar duplicados.
from utils.csv_manager import csv_loader

#Esta é responsável pelo tratamento dos dados do banco. Dentro do pacote db_manager existem as funções cep_loader, responsável
#por verificar se um registro já existe no BD e se não existir, fazer a consulta da localidade a partir do CEP.
#A função join_db_with_csv é responsável por fazer o join da localidade presente no DB, na planilha do BDO. Para tanto
#o valor do CEP é utilizado como chave única.
from utils import db_manager

In [2]:
caminho_do_csv = '../bdo_csv/bdo_report_16-02-2021-16-55-00.csv'
ceps_csv = csv_loader(caminho_do_csv)

Salva o csv localmente. Será utilizado como entrada para buscar os CEPs

In [3]:
ceps_csv.to_csv('../data/ceps_sem_localidades.csv', sep=',', header=['CEP'], index=False)

### Data cleaning finished. Now, start to populate the db

Esta função cria o DB e/ou tabelas se não existirem. Depois, verifica se um CEP já existe na tabela, se não, busca na API externa.

In [ ]:
db_manager.cep_loader()

 - db_file -> Caminho para o db populado com as localidades. Por padrão, será criado na pasta '../data/cepsdb.db'
 - csv_file -> Caminho onde está salvo o relatório extraído do bdo, em formato csv.


In [4]:
db_file = '../data/cepsdb.db'
csv_file = '../bdo_csv/bdo_report_16-02-2021-16-55-00.csv'

df, nao_localizado = db_manager.join_db_with_csv(db_file, csv_file)

In [6]:
print(f'Os ceps não localizados são: {nao_localizado}\nSe estiver visualizando uma lista com valores iguais a "nan", \nsignifica que foram encontrados CEPs inválidos e por isso não foram associados.')

Os ceps não localizados são: ['nan', 'nan', 'nan']
Se estiver visualizando uma lista com valores iguais a "nan", 
significa que foram encontrados CEPs inválidos e por isso não foram associados.


In [5]:
df.head(5)

In [23]:
#Corrige formatação dos campos CEP e Localidade DNE para serem salvos no arquivo xlsx
df_copy = df.copy()
df_copy['CEP String'] = [', '.join(map(str, l)) for l in df_copy['CEP']]
df_copy['Localidade DNE String'] = [', '.join(map(str, l)) for l in df_copy['Localidade DNE']]
df_copy['CEP'] = df_copy['CEP String']
df_copy['Localidade DNE'] = df_copy['Localidade DNE String']
df_copy = df_copy.drop(columns = ['Localidade DNE String', 'CEP String'])

df = df_copy
df.head(5)

,Cod. Pareamento,Cod. UF,Sigla UF,Cod. Subarea,Nome Subarea,Cod. Municipio,Nome Municipio,Codigo Agencia,Nome Agencia,Cod. Setor,...,CEP Logradouro CNEFE,CEPs Face,Localidade Face,Alterar Logradouro para DNE?,Observaçao,SIAPE Alteração,Nome Alteraçao,Data_Alteraçao,Status,Filtro
0,181411,51,MT,510340300,CUIABÁ I,5103403200,Cuiabá I,5103403,CUIABÁ,510340305400003,...,78058000,"78055714, 78058000",NOVO PARAISO,SIM,"Em pesquisa ao Google Maps, DNE e cadastro do ...",2317283,Micael Etiene de Souza,2021-02-02 16:11:40,Analisado,False
1,181545,51,MT,510340300,CUIABÁ I,5103403200,Cuiabá I,5103403,CUIABÁ,510340305400007,...,78048000,78048000,PAIAGUAS,NAO,As localidades na pesquisa do DNE que aparecem...,3160972,Elaine Cristina Teixeira Pinto,2021-02-03 13:36:50,Analisado,False
2,181546,51,MT,510340300,CUIABÁ I,5103403200,Cuiabá I,5103403,CUIABÁ,510340305400007,...,78048000,78048000,PAIAGUAS,NAO,"Em consulta Google Maps, consta no DNE como Jo...",3160972,Elaine Cristina Teixeira Pinto,2021-02-03 14:08:39,Analisado,False
3,179777,51,MT,510340300,CUIABÁ I,5103403200,Cuiabá I,5103403,CUIABÁ,510340305400011,...,78048000,78048000,JARDIM ITAPUA PAIAGUAS,NAO,"Após consulta Google Maps, DNE e Consulta do C...",3160972,Elaine Cristina Teixeira Pinto,2021-02-03 15:12:10,Analisado,False
4,181424,51,MT,510340300,CUIABÁ I,5103403200,Cuiabá I,5103403,CUIABÁ,510340305400014,...,78048000,78048000,CATEDRAL PAIAGUAS,NAO,"Em consulta ao Google Maps, DNE e Consulta CNE...",3160972,Elaine Cristina Teixeira Pinto,2021-02-03 16:12:02,Analisado,False


# Filtrando dados de acordo com as regras de negócio

In [24]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.0f}'.format)

### Cria uma máscara booleana para filtrar valores onde o campo Alterar Logradouro para DNE seja True or False

In [39]:
mask = dict()
for index, row in df.iterrows():

    try:
        mask[index] = str(int(row['CEP Logradouro CNEFE'])) in row['CEP'].split(',')
    except:
        mask[index] = False

df['Filtro'] = pd.Series(mask)

(65, 35)

## Regras de negócio:

### Regra 1
* Retornar casos onde: $ CEP ~ Logradouro ~ CNEFE ⊂ CEP ~~ \& ~~ Alterar Logradouro para DNE? == NAO$

In [41]:
#filtra por CEPs_CNEFE contidos na lista de CEPs DNE
df_regra1 = df.loc[df['Filtro'] == True]

#Filtra pelo atributo Alterar Log... DNE == 'NAO'
df_regra1 = df_regra1.loc[df_regra1['Alterar Logradouro para DNE?'] == 'NAO']
df_regra1.shape

(65, 35)

In [44]:
#exporta dataframe em formato csv
df_copy1.to_excel('../data/regra1_cepcnefe_in_cepdne_alteradne_nao.xlsx')

### Regra 2
* Retornar casos onde: $ CEP ~ Logradouro ~ CNEFE ⊄ CEP ~~ \& ~~ Alterar Logradouro para DNE? == SIM$

In [46]:
#filtra por CEPs_CNEFE não contidos na lista de CEPs DNE
df_regra2 = df.loc[df['Filtro'] == False]

#Filtra pelo atributo Alterar Log... DNE == 'SIM'
df_regra2 = df_regra2.loc[df_regra2['Alterar Logradouro para DNE?'] == 'SIM']
df_regra2.shape

(1373, 35)

In [49]:
#exporta dataframe em formato xlsx
df_regra2.to_excel('../data/regra2_cepcnefe_not_in_cepdne_alteradne_sim.xlsx')

## Verifica por que a localidade morada da serra é a única salva como string ao invés de lista